In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
! pip install transformers==4.0.1

In [3]:
!pip install torch==1.4.0

In [4]:
import torch
import random
import numpy as np
config = {
    'train_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/train.csv',
    'test_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/test.csv',
    'train_val_ratio': 0.1,

    # ------------- part 1 --------------- #
    # 'model_path': 'data/data94445',
    # ------------- part 1 ----------------#
    
    # ------------- part 2 --------------- #
    'head': 'cnn',
    # ------------- part 2 ----------------#
    
    # ------------- part 3 --------------- #
    'model_path': '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model',
    # ------------- part 3 ----------------#

    'batch_size': 16,   
    'num_epochs': 1,
    'learning_rate': 2e-5,
    'logging_step': 500,
    'seed': 2021}

config['device']='cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2021

In [5]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [6]:
torch.cuda.is_available()

False

In [7]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
def read_data(config, tokenizer, mode='train'):
    # 读取训练文件/测试文件
    data_df = pd.read_csv(config[f'{mode}_file_path'], sep=',')
    if mode == 'train':
        # 如果是训练文件 则划分训练集、验证集
        X_train, y_train = defaultdict(list), []
        X_val, y_val = defaultdict(list), []
        num_val = int(len(data_df) * config['train_val_ratio'])
    else:
        # 否则，只生成测试集
        X_test, y_test = defaultdict(list), []

    # 遍历文件中的美一行
    for i, row in tqdm(data_df.iterrows(), desc=f'preprocess {mode} data', total=len(data_df)):
        # 得到每个句子的标签 
        label=row[1] if mode == 'train' else 0
        # 得到每个句子
        sentence = row[-1]

        # ---------------------- part 1 ---------------------- #
        # add_special_tokens 添加一些特殊的toke： CLS、 SEP
        # return_token_type_ids 返回每个词所对应的id
        # return_attention_mask 批量时， padding部分无需注意
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, return_token_type_ids=True, return_attention_mask=True)

        # return：
        # input_ids；
        # token_type_ids
        # attention_mask
        # ---------------------- part 1 ---------------------- #

        if mode == 'train':
            # 当读取训练文件时
            if i < num_val:
                X_val['inputs_ids'].append(inputs['input_ids'])
                y_val.append(label)
                
                # ---------------------- part 1 ---------------------- #
                X_val['token_type_ids'].append(inputs['token_type_ids'])
                X_val['attention_mask'].append(inputs['attention_mask'])
                # ---------------------- part 1 ---------------------- #
                                
            else:
                X_train['inputs_ids'].append(inputs['input_ids'])
                y_train.append(label)

                # ---------------------- part 1 ---------------------- #
                X_train['token_type_ids'].append(inputs['token_type_ids'])
                X_train['attention_mask'].append(inputs['attention_mask'])
                # ---------------------- part 1 ---------------------- #

        else:
            X_test['inputs_ids'].append(inputs['input_ids'])
            y_test.append(label) 
            # ---------------------- part 1 ---------------------- #
            X_test['token_type_ids'].append(inputs['token_type_ids'])
            X_test['attention_mask'].append(inputs['attention_mask'])
            # ---------------------- part 1 ---------------------- #
            
    # 返回
    if mode == 'train':
        label2id = {label: i for i, label in enumerate(np.unique(y_train))} 
        id2label = {i: label for label, i in label2id.items()} 
        y_train = torch.tensor([label2id[i] for i in y_train], dtype=torch.long)  
        y_val = torch.tensor([label2id[i] for i in y_val], dtype=torch.long)  
        return X_train, y_train, X_val, y_val, label2id, id2label
        
    else:
        y_test = torch.tensor(y_test, dtype=torch.long)
        return X_test, y_test

In [8]:
from torch.utils.data import Dataset
class TNEWSData(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __getitem__(self, idx):
        return {
            'inputs_ids' : self.x['inputs_ids'][idx],
            'label' : self.y[idx],
            # ---------------------- part 1 ---------------------- #
            'token_type_ids': self.x['token_type_ids'][idx],
            'attention_mask': self.x['attention_mask'][idx]
            # ---------------------- part 1 ---------------------- #
        }
    
    def __len__(self):
        return self.y.size(0)

In [9]:
def collate_fn(examples):
    # 从 TNEWData 中返回的{}中，多了两个key: token_type_ids, attention_mask
    input_ids_list = []
    labels = []
    # ---------------------- part 1 ---------------------- #
    token_type_ids_list = []
    attention_mask_list = []
    # ---------------------- part 1 ---------------------- #

    for example in examples:
        #遍历样本， 我们先把句子放入列表中
        input_ids_list.append(example['inputs_ids'])
        labels.append(example['label'])
        # ---------------------- part 1 ---------------------- #
        token_type_ids_list.append(example['token_type_ids'])
        attention_mask_list.append(example['attention_mask'])
        # ---------------------- part 1 ---------------------- #
    
    # 对齐放入tensor中
    max_length = max(len(input_ids) for input_ids in input_ids_list)
    # shape (len(labels), max_length)
    input_ids_tensor = torch.zeros((len(labels), max_length), dtype=torch.long)

    # ---------------------- part 1 ---------------------- #
    token_type_ids_tensor = torch.zeros_like(input_ids_tensor)
    attention_mask_tensor = torch.zeros_like(input_ids_tensor)
    # ---------------------- part 1  ---------------------- #

    # 把列表中的数据放入tensor里
    for i, input_ids in enumerate(input_ids_list):
        input_ids_tensor[i, :len(input_ids)] = torch.tensor(input_ids, dtype=torch.long)
        # ---------------------- part 1 ---------------------- #
        # 第i个句子，填充 seq_len 这么长
        token_type_ids_tensor[i, :len(input_ids)] = torch.tensor(token_type_ids_list[i], dtype=torch.long)
        attention_mask_tensor[i, :len(input_ids)] = torch.tensor(attention_mask_list[i], dtype=torch.long)
        # ---------------------- part 1 ---------------------- #
    
    return{
        'input_ids' : input_ids_tensor,
        'labels' : torch.tensor(labels, dtype=torch.long),
        # ---------------------- part 1 ---------------------- #
        'token_type_ids': token_type_ids_tensor,
        'attention_mask': attention_mask_tensor
        # ---------------------- part 1 ---------------------- #
    }

In [10]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
def build_dataloader(config):
    # ---------------------- part 1 ---------------------- #
    # 加载词表
    tokenizer = BertTokenizer.from_pretrained(config['model_path'])
    # ---------------------- part 1 ---------------------- #
    X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, tokenizer, mode='train')
    X_test, y_test = read_data(config, tokenizer, mode='test')

    train_dataset = TNEWSData(X_train, y_train)
    val_dataset = TNEWSData(X_val, y_val)
    test_dataset = TNEWSData(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collate_fn)

    return train_dataloader, val_dataloader, test_dataloader, id2label

In [11]:
train_dataloader, val_dataloader, test_dataloader, id2label = build_dataloader(config)

preprocess test data: 100%|██████████| 10000/10000 [00:08<00:00, 1225.94it/s]


In [12]:
for batch in train_dataloader:
    print(batch)
    break

{'input_ids': tensor([[  101,  6205,  2128,  4415,  2339,  1920,  2110,  2247,   754,   784,
           720,  2231,  3613,  4638,  1920,  2110,  8043,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  1963,   862,  6397,   817,  4906,  3683,  2456,  6379,  6205,
          5885,  3172,  3121,  1359,  2832,  5074,  2013,  1232,  4638,   752,
          2658,  8043,   102,     0,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  8391,  4413,  3215,  2853,  4170,  6443,  3297,  7464,  3698,
          8043,  2001,  3209,  6804,  2802,  3952,  2767,  6804,  2853,  4170,
          8024, 12695,  1402,  4170,  1008,  1613,  4125,   102,     0,     0,
             0],
        [  101,  1395,  1164,  9101,  2797,  5632,   671,   860,  4638,  2797,
          1220,  2913,  3221,  2582,   720,  4500,  4638,  8043,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,  

In [13]:
from sklearn.metrics import f1_score
def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss = 0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))

    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['labels'])
            batch = {item: value.to(config['device']) for item, value in batch.items()}
            loss, logits = model(**batch)[:2]

            val_loss += loss.item()
            preds.append(logits.argmax(dim=-1).detach().cpu())

    avg_val_loss = val_loss / len(val_dataloader)
    labels = torch.cat(labels, dim=0).numpy()
    preds = torch.cat(preds, dim=0).numpy()
    f1 = f1_score(labels, preds, average='macro')
    return avg_val_loss, f1

In [14]:
# Bert_model train
from transformers import BertConfig, BertForSequenceClassification
from transformers import AdamW
from tqdm import trange
def train(config, id2label, train_dataloader, val_dataloader):
    # ---------------------- part 1 ---------------------- #
    # 配置文件
    bert_config = BertConfig.from_pretrained(config['model_path'])
    bert_config.num_labels = len(id2label)
    model = BertForSequenceClassification.from_pretrained(config['model_path'], config=bert_config)
    # ---------------------- part 1 ---------------------- #

    # 定义优化器
    optimizer = AdamW(model.parameters(), lr=config['learning_rate'])

    # 有GPU， 将模型放入GPU
    model.to(config['device'])
    epoch_iterator = trange(config['num_epochs'])
    global_steps = 0
    train_loss = 0.
    logging_loss = 0.

    #一共跑了多少个迭代
    for epoch in epoch_iterator:

        train_iterator = tqdm(train_dataloader, desc='Training', total=len(train_dataloader))
        model.train()
        # 遍历每个batch
        for batch in train_iterator:
            # 字典中的 value 送入GPU
            batch = {item: value.to(config['device']) for item, value in batch.items()}
            
            # 将字典作为关键字参数传递给Python中的函数
            loss = model(**batch)[0]
            
            # 模型中的参数梯度设为0
            model.zero_grad()
            
            # 反向传播
            loss.backward()
            
            # 更新参数
            optimizer.step()

            # 叠加loss
            train_loss += loss.item()
            global_steps += 1

            if global_steps % config['logging_step'] == 0:
                print_train_loss = (train_loss - logging_loss) / config['logging_step']
                logging_loss = train_loss

                avg_val_loss, f1 = evaluation(config, model, val_dataloader)

                print_log = f'>>> training loss: {print_train_loss:.4f}, valid loss: {avg_val_loss:.4f}, ' \
                            f'valid f1 score: {f1:.4f}'
                print(print_log)
                model.train()

    return model

In [15]:
model = train(config, id2label, train_dataloader, val_dataloader)

Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/dataset/BERT_model were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

>>> training loss: 1.6373, valid loss: 1.4193, valid f1 score: 0.4969



Training:  33%|███▎      | 999/3002 [2:20:12<3:58:59,  7.16s/it]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:03<18:16,  3.29s/it]

Evaluation:   1%|          | 2/334 [00:05<14:48,  2.68s/it]

Evaluation:   1%|          | 3/334 [00:08<14:21,  2.60s/it]

Evaluation:   1%|          | 4/334 [00:10<13:23,  2.44s/it]

Evaluation:   1%|▏         | 5/334 [00:12<12:51,  2.35s/it]

Evaluation:   2%|▏         | 6/334 [00:14<12:05,  2.21s/it]

Evaluation:   2%|▏         | 7/334 [00:16<11:38,  2.14s/it]

Evaluation:   2%|▏         | 8/334 [00:18<11:18,  2.08s/it]

Evaluation:   3%|▎         | 9/334 [00:20<11:05,  2.05s/it]

Evaluation:   3%|▎         | 10/334 [00:22<11:42,  2.17s/it]

Evaluation:   3%|▎         | 11/334 [00:24<11:48,  2.19s/it]

Evaluation:   4%|▎         | 12/334 [00:26<11:24,  2.13s/it]

Evaluation:   4%|▍         | 13/334 [00:29<11:45,  2.20s/it]

Evaluation:   4%|▍         | 14/334 [00:31<11:20,  2.13s/it]

Evaluation:   4%|▍   

>>> training loss: 1.4121, valid loss: 1.3300, valid f1 score: 0.5082



Training:  50%|████▉     | 1499/3002 [3:35:49<3:19:04,  7.95s/it]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:03<18:56,  3.41s/it]

Evaluation:   1%|          | 2/334 [00:05<14:51,  2.68s/it]

Evaluation:   1%|          | 3/334 [00:08<14:22,  2.61s/it]

Evaluation:   1%|          | 4/334 [00:10<13:28,  2.45s/it]

Evaluation:   1%|▏         | 5/334 [00:12<13:01,  2.38s/it]

Evaluation:   2%|▏         | 6/334 [00:14<12:15,  2.24s/it]

Evaluation:   2%|▏         | 7/334 [00:16<11:44,  2.16s/it]

Evaluation:   2%|▏         | 8/334 [00:18<11:21,  2.09s/it]

Evaluation:   3%|▎         | 9/334 [00:20<11:07,  2.05s/it]

Evaluation:   3%|▎         | 10/334 [00:22<11:48,  2.19s/it]

Evaluation:   3%|▎         | 11/334 [00:25<11:51,  2.20s/it]

Evaluation:   4%|▎         | 12/334 [00:27<11:24,  2.13s/it]

Evaluation:   4%|▍         | 13/334 [00:29<11:43,  2.19s/it]

Evaluation:   4%|▍         | 14/334 [00:31<11:19,  2.12s/it]

Evaluation:   4%|▍  

...


Training:  83%|████████▎ | 2499/3002 [6:05:57<52:04,  6.21s/it]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:02<15:50,  2.86s/it]

Evaluation:   1%|          | 2/334 [00:04<12:30,  2.26s/it]

Evaluation:   1%|          | 3/334 [00:06<12:13,  2.22s/it]

Evaluation:   1%|          | 4/334 [00:08<11:18,  2.06s/it]

Evaluation:   1%|▏         | 5/334 [00:10<10:48,  1.97s/it]

Evaluation:   2%|▏         | 6/334 [00:12<10:22,  1.90s/it]

Evaluation:   2%|▏         | 7/334 [00:14<10:06,  1.85s/it]

Evaluation:   2%|▏         | 8/334 [00:15<09:59,  1.84s/it]

Evaluation:   3%|▎         | 9/334 [00:17<09:52,  1.82s/it]

Evaluation:   3%|▎         | 10/334 [00:19<10:13,  1.90s/it]

Evaluation:   3%|▎         | 11/334 [00:21<10:10,  1.89s/it]

Evaluation:   4%|▎         | 12/334 [00:23<09:55,  1.85s/it]

Evaluation:   4%|▍         | 13/334 [00:25<10:04,  1.88s/it]

Evaluation:   4%|▍         | 14/334 [00:27<09:55,  1.86s/it]

Evaluation:   4%|▍    

>>> training loss: 1.2811, valid loss: 1.2828, valid f1 score: 0.4986



Training: 100%|█████████▉| 2999/3002 [7:13:27<00:19,  6.55s/it]

Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/334 [00:02<15:20,  2.76s/it]

Evaluation:   1%|          | 2/334 [00:04<12:17,  2.22s/it]

Evaluation:   1%|          | 3/334 [00:06<12:10,  2.21s/it]

Evaluation:   1%|          | 4/334 [00:08<11:16,  2.05s/it]

Evaluation:   1%|▏         | 5/334 [00:10<10:44,  1.96s/it]

Evaluation:   2%|▏         | 6/334 [00:12<10:22,  1.90s/it]

Evaluation:   2%|▏         | 7/334 [00:13<10:06,  1.85s/it]

Evaluation:   2%|▏         | 8/334 [00:15<09:56,  1.83s/it]

Evaluation:   3%|▎         | 9/334 [00:17<09:47,  1.81s/it]

Evaluation:   3%|▎         | 10/334 [00:19<10:14,  1.90s/it]

Evaluation:   3%|▎         | 11/334 [00:21<10:14,  1.90s/it]

Evaluation:   4%|▎         | 12/334 [00:23<10:00,  1.86s/it]

Evaluation:   4%|▍         | 13/334 [00:25<10:09,  1.90s/it]

Evaluation:   4%|▍         | 14/334 [00:27<09:55,  1.86s/it]

Evaluation:   4%|▍    

>>> training loss: 1.2837, valid loss: 1.2428, valid f1 score: 0.5326



100%|██████████| 1/1 [7:24:23<00:00, 26663.35s/it]


In [16]:
def predict(config, id2label, model, test_dataloader):
    test_iterator = tqdm(test_dataloader, desc='Predicting', total=len(test_dataloader))
    model.eval()
    test_preds = []
    with torch.no_grad():
        for batch in test_iterator:
            batch = {item: value.to(config['device']) for item, value in batch.items()}
            logits = model(**batch)[1]

            test_preds.append(logits.argmax(dim=-1).detach().cpu())
    test_preds = torch.cat(test_preds, dim=0).numpy()
    test_preds = [id2label[id_] for id_ in test_preds]
    test_df = pd.read_csv(config['test_file_path'], sep=',')
    test_df.insert(1, column='label', value=test_preds)
    test_df.drop(columns=['sentence'], inplace=True)
    test_df.to_csv('submission.csv', index=False, encoding='utf8')

In [17]:
predict(config, id2label, model, test_dataloader)

Predicting: 100%|██████████| 625/625 [18:49<00:00,  1.81s/it]
